In [1]:
!pip install biopython
!pip install fair-esm
!pip install torch
import torch
import pandas as pd
import numpy as np
from Bio import SeqIO
import esm
import gc
# CUDA
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available')
else:
    device = torch.device('cpu')
    print('CPU available')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
# 加载模型和字母表
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
batch_converter = alphabet.get_batch_converter()
model = model.to(device)
model.eval()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 2560, padding_idx=1)
  (layers): ModuleList(
    (0-35): 36 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
        (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
        (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
        (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=2560, out_features=10240, bias=True)
      (fc2): Linear(in_features=10240, out_features=2560, bias=True)
      (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=1440, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((2560,), ep

In [3]:
def esm_to_feature(tuple_seq_list, num_layer, device):
    i = 1
    embeddings = []
    for tuple_seq in tuple_seq_list:
        print(f"{i}/{len(tuple_seq_list)}")
        i += 1
        print('长度: ', len(tuple_seq[1]))
        print(tuple_seq[1]) # tuple_seq是一个tuple
        # 将蛋白质序列列表加载到批处理转换器中，且批处理转换器接收的是一个list变量
        batch_labels, batch_strs, batch_tokens = batch_converter([tuple_seq])
        batch_tokens = batch_tokens.to(device)  # 将批处理令牌移动到所选设备上

        # 提取每个残基的表示
        with torch.no_grad():
            # repr_layers 等于模型层数
            results = model(batch_tokens, repr_layers=[num_layer], return_contacts=False)
        token_representations = results["representations"][num_layer]

        batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)  # 记录每条序列的真实长度+2 (开始和结束标记符)
        # 通过平均值生成每个序列的表示
        # 注意：令牌0始终是序列开始令牌，所以第一个残基的令牌是1。
        sequence_representation = token_representations[0, 1:batch_lens[0] - 1].mean(0)
        embeddings.append(sequence_representation)

        del batch_tokens, results, token_representations, batch_labels, batch_strs
        torch.cuda.empty_cache()  # 清理 GPU 缓存
        gc.collect()  # 释放 Python 内存

    return torch.stack(embeddings)

In [8]:
def format(file_path):
    sequences = []
    labels = []
    for record in SeqIO.parse(file_path, "fasta"):
        seq = str(record.seq)
        if len(seq) > 5000:
            seq = seq[:5000]
        sequences.append(tuple([_, seq]))
        labels.append(0)
    return sequences, labels

In [9]:
file_path = r'merged_test.fasta'
tuple_seq_list, _ = format(file_path)
embeddings = esm_to_feature(tuple_seq_list, num_layer=36, device=device)
torch.save(embeddings, r'merged_test_esm2_t36.pt')
print('done')

1/478
长度:  440
MNYSLHLAFVCLSLFTERMCIQGSQFNVEVGRSDKLSLPGFENLTAGYNKFLRPNFGGEPVQIALTLDIASISSISESNMDYTATIYLRQRWMDQRLVFEGNKSFTLDARLVEFLWVPDTYIVESKKSFLHEVTVGNRLIRLFSNGTVLYALRITTTVACNMDLSKYPMDTQTCKLQLESWGYDGNDVEFTWLRGNDSVRGLEHLRLAQYTIERYFTLVTRSQQETGNYTRLVLQFELRRNVLYFILETYVPSTFLVVLSWVSFWISLDSVPARTCIGVTTVLSMTTLMIGSRTSLPNTNCFIKAIDVYLGICFSFVFGALLEYAVAHYSSLQQMAAKDRGTTKEVEEVSITNIINSSISSFKRKISFASIEISSDNVDYSDLTMKTSDKFKFVFREKMGRIVDYFTIQNPSNVDHYSKLLFPLIFMLANVFYWAYYMYF
2/478
长度:  1248
MDAKARNCLLQHREALEKDIKTSYIMDHMISDGFLTISEEEKVRNEPTQQQRAAMLIKMILKKDNDSYVSFYNALLHEGYKDLAALLHDGIPVVSSSSGKDSVSGITSYVRTVLCEGGVPQRPVVFVTRKKLVNAIQQKLSKLKGEPGWVTIHGMAGCGKSVLAAEAVRDHSLLEGCFPGGVHWVSVGKQDKSGLLMKLQNLCTRLDQDESFSQRLPLNIEEAKDRLRILMLRKHPRSLLILDDVWDSWVLKAFDSQCQILLTTRDKSVTDSVMGPKYVVPVESSLGKEKGLEILSLFVNMKKADLPEQAHSIIKECKGSPLVVSLIGALLRDFPNRWEYYLKQLQNKQFKRIRKSSSYDYEALDEAMSISVEMLREDIKDYYTDLSILQKDVKVPTKVLCILWDMETEEVEDILQEFVNKSLLFCDRNGKSFRYYLHDLQVDFLTEKNCSQLQDLHKKIITQFQRYHQPHTLSPDQEDCMYWYNFLAYHMASAKMHKELCALMFSLDWIKAKTELVGPAHLIHEFVE

KeyboardInterrupt: 

In [10]:
file_path = r'merged_train.fasta'
tuple_seq_list, _ = format(file_path)
embeddings = esm_to_feature(tuple_seq_list, num_layer=36, device=device)
torch.save(embeddings, r'merged_independent_esm2_t36.pt')
print('done')

1/2063
长度:  403
MQRAVSVVARLGFRLQAFPPALCRPLSCAQEVLRRTPLYDFHLAHGGKMVAFAGWSLPVQYRDSHTDSHLHTRQHCSLFDVSHMLQTKILGSDRVKLMESLVVGDIAELRPNQGTLSLFTNEAGGILDDLIVTNTSEGHLYVVSNAGCWEKDLALMQDKVRELQNQGRDVGLEVLDNALLALQGPTAAQVLQAGVADDLRKLPFMTSAVMEVFGVSGCRVTRCGYTGEDGVEISVPVAGAVHLATAILKNPEVKLAGLAARDSLRLEAGLCLYGNDIDEHTTPVEGSLSWTLGKRRRAAMDFPGAKVIVPQLKGRVQRRRVGLMCEGAPMRAHSPILNMEGTKIGTVTSGCPSPSLKKNVAMGYVPCEYSRPGTMLLVEVRRKQQMAVVSKMPFVPTNYYTLK
2/2063
长度:  446
MRTLNTSAMDGTGLVVERDFSVRILTACFLSLLILSTLLGNTLVCAAVIRFRHLRSKVTNFFVISLAVSDLLVAVLVMPWKAVAEIAGFWPFGSFCNIWVAFDIMCSTASILNLCVISVDRYWAISSPFRYERKMTPKAAFILISVAWTLSVLISFIPVQLSWHKAKPTSPSDGNATSLAETIDNCDSSLSRTYAISSSVISFYIPVAIMIVTYTRIYRIAQKQIRRIAALERAAVHAKNCQTTTGNGKPVECSQPESSFKMSFKRETKVLKTLSVIMGVFVCCWLPFFILNCILPFCGSGETQPFCIDSNTFDVFVWFGWANSSLNPIIYAFNADFRKAFSTLLGCYRLCPATNNAIETVSINNNGAAMFSSHHEPRGSISKECNLVYLIPHAVGSSEDLKKEEAAGIARPLEKLSPALSVILDYDTDVSLEKIQPITQNGQHPT
3/2063
长度:  640
MHVKKYLLKGLHRLQKGPGYTYKELLVWYCDNTNTHGPKRIICEGPKKKAMWFLLTLLFAALVCWQWGIFIRTYLSWEVSVSLSVGFKTMDFPAVTICNAS

KeyError: '>'

In [6]:
file_path = r'negative_train_sequence.fasta'
tuple_seq_list, _ = format(file_path)
embeddings = esm_to_feature(tuple_seq_list, num_layer=36, device=device)
torch.save(embeddings, r'positive_independent_esm2_t36.pt')
print('done')

1/1059
长度:  51
MICCQETVYSCEYWRTFKRYFSSWRFKIRSWLTYLSWGLLCTSWYCKVKMS
2/1059
长度:  190
MSLPIFVFSFNLEHVFVFLLSLSFGLDFILLPFFSSFPSTEAVGKFSGLITRGILRSPRSFAFAMASDIIRLAFSCLLSDSTFFLCFFSDMTLTWVFCNLVYVPEPSLFLEDGGLPLGLFTEDIFGVRQRLALCFFTTPGSAQEKSIERRFLNGLFSKLQPRDAQPSTNTHFDLSTHFCLGKFISTVHPI
3/1059
长度:  323
MRDSACWSQRKDELLQQARKRFLNKSSEDDAASESFLPSEGASSDPVTLRRRMLAAARNGGFRSSRPPSAPLPSSAASCALCPTDWRRPVPILPLHGKAGLTALPLYKACGLIVFGQLINLILLCNTFYVTFLFPLETLQILTVGMISSGVDWTAWGGGRSGGSEPVACLQQAASTPASCIRPTNAGVLSTTPSGKSVGEAHSVSPPPRRGVTSVIKLLSLLWKHVDCARARPTGSCTPEQQGILEKELLVRYLEQRRGKSRAIGCDEVTPFCPTTSGTDFPSLQSKAGLISVNSGAPASHECAPWVPSPLSISLSRLDLGSG
4/1059
长度:  41
MDGSGEQPRGGVSSRIEQGEWGGRHPSWPWTRCLRMRPPRS
5/1059
长度:  57
MLSIDLQLSSICVPRMQLKTCYVEEIRGVVLEDRHLWNDSHPLKLGGWRPASLRSWG
6/1059
长度:  214
MVLSRGSLLNTRPRITSGSGSTGRPRSGSTCRSTDHEPGKLAKLQAQVRIGGKGTAHRKKKVFHRTATADDKKLQFSLKKLQVNNISGIEKVNMFTNQGTVIHFNNPKFQASLAVNTFTITGHAEAKQVTEMLPSVLSQLGADSLTSLRRLAEVLPKQPVDGKAPLATGGDDDDGVPELWRILMRLPGMRQAELSRLLKKIKLEEVTGSCYFLL
7/1059
长度:  104
METSHAHES

KeyboardInterrupt: 

In [ ]:
file_path = r'positive_train.fasta'
tuple_seq_list, _ = format(file_path)
embeddings = esm_to_feature(tuple_seq_list, num_layer=36, device=device)
torch.save(embeddings, r'positive_train_esm2_t36.pt')
print('done')

1/1224
长度:  403
MQRAVSVVARLGFRLQAFPPALCRPLSCAQEVLRRTPLYDFHLAHGGKMVAFAGWSLPVQYRDSHTDSHLHTRQHCSLFDVSHMLQTKILGSDRVKLMESLVVGDIAELRPNQGTLSLFTNEAGGILDDLIVTNTSEGHLYVVSNAGCWEKDLALMQDKVRELQNQGRDVGLEVLDNALLALQGPTAAQVLQAGVADDLRKLPFMTSAVMEVFGVSGCRVTRCGYTGEDGVEISVPVAGAVHLATAILKNPEVKLAGLAARDSLRLEAGLCLYGNDIDEHTTPVEGSLSWTLGKRRRAAMDFPGAKVIVPQLKGRVQRRRVGLMCEGAPMRAHSPILNMEGTKIGTVTSGCPSPSLKKNVAMGYVPCEYSRPGTMLLVEVRRKQQMAVVSKMPFVPTNYYTLK
2/1224
长度:  1000
MAESASPPSSSAAAPAAEPGVTTEQPGPRSPPSSPPGLEEPLDGADPHVPHPDLAPIAFFCLRQTTSPRNWCIKMVCNPWFECVSMLVILLNCVTLGMYQPCDDMDCLSDRCKILQVFDDFIFIFFAMEMVLKMVALGIFGKKCYLGDTWNRLDFFIVMAGMVEYSLDLQNINLSAIRTVRVLRPLKAINRVPSMRILVNLLLDTLPMLGNVLLLCFFVFFIFGIIGVQLWAGLLRNRCFLEENFTIQGDVALPPYYQPEEDDEMPFICSLSGDNGIMGCHEIPPLKEQGRECCLSKDDVYDFGAGRQDLNASGLCVNWNRYYNVCRTGSANPHKGAINFDNIGYAWIVIFQVITLEGWVEIMYYVMDAHSFYNFIYFILLIIVGSFFMINLCLVVIATQFSETKQREHRLMLEQRQRYLSSSTVASYAEPGDCYEEIFQYVCHILRKAKRRALGLYQALQSRRQALGPEAPAPAKPGPHAKEPRHYHGKTKGQGDEGRHLGSRHCQTLHGPASPGNDHSGRELCPQHSPLDATPHTLVQPIPATLASDPASCPCCQHEDGRR